In [1]:
from bs4 import BeautifulSoup
import requests






url = "https://www.ime.usp.br/bioinfo/pos/teses-dissertacoes"
page = requests.get(url)

soup = BeautifulSoup(page.text, "html.parser")

In [2]:
list_of_thesis = soup.find(class_ = "striped-table")

all_links = list_of_thesis.find_all("a")

In [3]:
list_of_pre_thesis_urls = []

for link in all_links:
    link_now = link.get('href')
    if "teses" in link_now:
        list_of_pre_thesis_urls.append(link_now)

In [6]:
list_of_pre_thesis_urls[0:5]

['http://teses.usp.br/autor.php?autor=57A9A0FBF285',
 'http://teses.usp.br/autor.php?autor=17A7188B7471',
 'http://teses.usp.br/autor.php?autor=17A86D22C7A1',
 'http://teses.usp.br/autor.php?autor=B7AAB42FA53B',
 'http://teses.usp.br/autor.php?autor=97A5A5761BB9']

In [32]:
def get_thesis_url(pre_thesis_url):
    page = requests.get(pre_thesis_url)
    soup = BeautifulSoup(page.text, "html.parser")
    box = soup.find(class_="dadosLinha dadosCor1")
    all_links = box.find_all("a")
    thesis_url = all_links[0].get("href")
    return(thesis_url)

def get_comittee_members (thesis_url):
    page = requests.get(thesis_url)
    soup = BeautifulSoup(page.text, "html.parser")
    commitee = soup.select("#CorpoTexto > div:nth-child(23)")[0].text
    commitee = commitee.split("\n\n")
    student = soup.select("#CorpoTexto > div:nth-child(9)")[0].text
    students_commitee_dict = dict()
    students_commitee_dict[student] = commitee
    return(students_commitee_dict)


In [37]:
bioinfo_commitees_dict = dict()

for pre_thesis_url in list_of_pre_thesis_urls:
    try:
        thesis_url = get_thesis_url(pre_thesis_url)
        commitee_dict = get_comittee_members(thesis_url)
        bioinfo_commitees_dict.update(commitee_dict)
    except:
        None

In [50]:
import pandas as pd

bioinfo_commitees_df = pd.DataFrame.from_dict(bioinfo_commitees_dict, orient = "index")

bioinfo_commitees_df

,0,1,2,3,4,5
Junier Marrero Gutierrez,,"Koide, Tie (Presidente)","Martinez, Cristina Elisa Alvarez","Pinheiro, Daniel Guariz","Silva Neto, José Freire da\n",None
Caio Rafael do Nascimento Santiago,,"Digiampietri, Luciano Antonio (Presidente)","Chaim, Marcos Lordello","Ferreira, João Eduardo\n",None,None
Suzane de Andrade Barboza,,"Digiampietri, Luciano Antonio (Presidente)","Amaral, Fábio Sarubbi Raposo do","Lima, Ariane Machado","Oliveira, Julio Cezar Franco de\n",None
Juan Enrique Faya Castillo,,"Oliveira, Paulo Sérgio Lopes de (Presidente)","Benedetti, Celso Eduardo","Giuliatti, Silvana","Schechtman, Deborah\n",None
Felipe Prata Lima,,"Setubal, João Carlos (Presidente)","Araujo, Welington Luiz de","Dias Neto, Emmanuel","Hashimoto, Ronaldo Fumio","Varani, Alessandro de Mello\n"
...,...,...,...,...,...,...
Gustavo Henrique Esteves,,"Neves, Eduardo Jordao (Presidente)","Almeida, Sergio Verjovski de","Cesar Junior, Roberto Marcondes","Garcia, Nancy Lopes","Nunes, Luiz Roberto\n"
Bruno Monte de Castro,,"Leonardi, Florencia Graciela (Presidente)","García, Jesús Enrique","Iambartsev, Anatoli\n",None,None
Cesar Armando Beltran Castañon,,"Gruber, Arthur (Presidente)","Bruno, Odemir Martinez","Ferreira, Marcelo Urbano","Hirata, Nina Sumiko Tomita","Lotufo, Roberto de Alencar\n"
Bruno Sanchez de Araujo,,"Machado-Lima, Ariane (Presidente)","Martins Junior, David Corrêa","Paraboni, Ivandre","Salvini, Rogerio Lopes\n",None


False

NameError: name 'which' is not defined

{'Junier Marrero Gutierrez': ['',
  'Koide, Tie (Presidente)',
  'Martinez, Cristina Elisa Alvarez',
  'Pinheiro, Daniel Guariz',
  'Silva Neto, José Freire da\n']}